In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
import docx
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import pyttsx3
import speech_recognition as sr
import openpyxl

load_dotenv() # this function loads the env variable into the environment

# 2.Function to extract text from PDF files
@st.cache_data(show_spinner=False) # It caches the result of the get_pdf_text     #it will return the cached result
def get_pdf_text(pdf_docs):
    text = ""                 # empty string will accumlate the extracted text from the pdf
    for pdf in pdf_docs:      # iterates over all the pdf's in the pdf list
        pdf_reader = PdfReader(pdf) # creates an object, which reads the content in the pdf 
        for page in pdf_reader.pages: #iterates throgh all the pages
            text += page.extract_text() #extract_text() method is called on each page to get the text content of the page
    return text

# 3.Function to extract text from Word files
@st.cache_data(show_spinner=False) # It caches the result of the get_docx_text   #it will return the cached result
def get_docx_text(docx_files): #docx_file is list of Word document file objects.
    text = ""
    for doc in docx_files: #This loop iterates over each Word document in the docx_files list.
        doc_reader = docx.Document(doc)  # docx is module provided by python library
        for para in doc_reader.paragraphs: # This loop iterates over each paragraph in the current Word document.
            text += para.text + "\n" 
    return text

# 4.Function to extract text from Excel files
@st.cache_data(show_spinner=False) # decorator is used to cache the result of the get_excel_text function using Streamlit.
def get_excel_text(excel_files):
    text = ""
    for excel in excel_files: #This loop iterates over each Excel file in the excel_files list.
        df = pd.read_excel(excel)
        text += df.to_string(index=False) #DataFrame is called to convert the DataFrame to a string representation.
    return text

# 5.Function to split text into chunks
@st.cache_data(show_spinner=False) #This decorator is used to cache the result of the get_text_chunks function using Streamlit.
def get_text_chunks(text): #string containing the text to be split into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000) #object is created 
    chunks = text_splitter.split_text(text) # object is called with the input text
    return chunks  #The function returns the list of text chunks.

# 6.Function to create vector store using FAISS
@st.cache_data(show_spinner=False)
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") #This object is used to generate embeddings for the text chunks. #specifies the model used to generate the embeddings.
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)# FAISS  vector store from the text chunks using the provided embeddings.
    vector_store.save_local("faiss_index") #saves the created FAISS vector store to a local file named faiss_index # doesnt have to recompute.

# 7.Function to load conversational chain for question answering
def get_conversational_chain():
    prompt_template = """ 
    Answer the question as detailed as possible from the provided context. If the answer is not in
    the provided context, just say, "Answer is not available in the context." Do not provide a wrong answer.\n\n
    Context:\n {context}\n
    Question: \n{question}\n
    Answer:
    """
# variable contains multiple string and as placeholders where if fill the context and questions dynamically when the function is called.
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3) #instance is created with specified
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])#object is created with prompt template and list of input variables.
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)# function is called with parameters
    return chain #The function returns the loaded conversational chain

# 8.Function to process user input question and retrieve answer
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")# instance is created with specified model #This object is used to generate embeddings (numerical representations) for text.
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question) #performs similarity search
    chain = get_conversational_chain() #function is called to retrieve a conversational chain object.
    response = chain.invoke({"input_documents": docs, "question": user_question}, return_only_outputs=True)#only the outputs (responses) from the conversational chain should be returned.
    return response["output_text"] 

# 9.Function to convert text to speech using pyttsx3
def text_to_speech(text, mute=False): #function is taking a parameter mute=false(default)
    if not mute: #conditional check
        engine = pyttsx3.init() #initializing pyttsx3 #This engine handles the conversion process.
        engine.setProperty('rate', 150)  # Speed percent (can go over 100)
        engine.setProperty('volume', 1.0)  # Volume 0-1
        engine.say(text)# instructs the engine to convert text to speech.
        engine.runAndWait() #initiates the speech synthesis and waits for the conversion to complete before proceeding.

# 10.Function to handle voice commands
def get_voice_command():
    recognizer = sr.Recognizer() # creates a Recognizer instance from the SpeechRecognition library.
    microphone = sr.Microphone() # creates a Microphone instance for capturing audio input.

    with microphone as source: # sets up the microphone as the audio source for listening.
        st.info("Listening for your question...") # displays an informational message indicating that the function is ready to listen for voice input.
        audio = recognizer.listen(source) #captures audio from the microphone (source) and stores it in the audio variable.
    try:
        command = recognizer.recognize_google(audio)#attempts to recognize speech from the captured audio using Google's speech recognition service.
        st.success(f"You said: {command}")
        return command
    except sr.UnknownValueError: #handles cases where speech recognition could not understand the audio input
        st.error("Sorry, I did not understand that.")
        return ""
    except sr.RequestError: #handles cases where the speech recognition service is unavailable (RequestError).
        st.error("Could not request results; check your network connection.")
        return ""#The recognized command (command) is returned as a string.

# Main Streamlit application
def main():
    st.set_page_config(page_title="Document Chatbot", layout="wide")
    st.title("Document Chatbot")

    st.markdown(
        """
        <style>
        .main {
            background-color: #000000;
            padding: 2rem;
            font-family: Arial, sans-serif;
            color: #333;
        }
        .sidebar .sidebar-content {
            background-image: linear-gradient(#2E7D32,#1B5E20);
            color: white;
        }
        .stButton>button {
            color: white;
            background-color: #2E7D32;
            border: None;
            padding: 0.5rem 1rem;
            font-size: 1rem;
            border-radius: 8px;
        }
        .stButton>button:hover {
            background-color: #1B5E56;
            border: None;
        }
        .header {
            text-align: center;
            margin-bottom: 2rem;
        }
        .header h1 {
            font-size: 3rem;
            color: #2E7D32;
        }
        .header p {
            font-size: 1.2rem;
            color: #666;
        }
        .container {
            display: flex;
            flex-direction: column;
            align-items: center;
        }
        .question-input {
            width: 80%;
            margin-bottom: 1rem;
        }
        .response {
            width: 80%;
            padding: 1rem;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            margin-top: 1rem;
        }
        .response h3 {
            color: #2E7D32;
        }
        .speak-btn {
            background-color: #2E7D32; /* Green color */
            color: white;
            border: none;
            padding: 0.5rem 1rem;
            font-size: 1rem;
            border-radius: 8px;
            margin-top: 1rem;
        }
        .speak-btn:hover {
            background-color: #1B5E56; /* Darker green on hover */
        }
        .mute-btn {
            background-color: #FF5722; /* Orange color */
            color: white;
            border: none;
            padding: 0.5rem 1rem;
            font-size: 1rem;
            border-radius: 8px;
            margin-top: 1rem;
        }
        .mute-btn:hover {
            background-color: #E64A19; /* Darker orange on hover */
        }
        </style>
        """,
        unsafe_allow_html=True
    )

    st.markdown("""
    <div class="header">
        <h1> Chat with Dexter 🤖</h1>
        <p>Upload your documents, process them, and ask questions directly from the content.</p>
    </div>
    """, unsafe_allow_html=True)

    # File uploader for various document types
    doc_files = st.file_uploader("Upload your Documents", accept_multiple_files=True, type=["pdf", "docx", "xlsx"])
    #This line creates a file uploader widget in the Streamlit app.
    if doc_files: #checking uploaded file. 
        # Process documents and store vector store
        if st.button("Submit & Process"):
            with st.spinner("Processing..."):
                try:
                    raw_text = "" #Initializes an empty string to store the raw text extracted from documents.
                    pdf_docs = [doc for doc in doc_files if doc.type == "application/pdf"] # document type classification
                    docx_files = [doc for doc in doc_files if doc.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document"]
                    excel_files = [doc for doc in doc_files if doc.type == "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"]

                    if pdf_docs:
                        raw_text += get_pdf_text(pdf_docs) #If there are PDF documents, extracts text from them using get_pdf_text and appends to raw_text
                    if docx_files:
                        raw_text += get_docx_text(docx_files)
                    if excel_files:
                        raw_text += get_excel_text(excel_files)

                    text_chunks = get_text_chunks(raw_text) #Splits the raw_text into chunks using get_text_chunks
                    get_vector_store(text_chunks) #Creates a vector store from the text chunks using get_vector_store
                    st.success("Processing complete. You can now ask questions!")
                except Exception as e:
                    st.error(f"An error occurred: {e}")

        # State management for mute functionality
        if "mute" not in st.session_state:
            st.session_state.mute = False

        def toggle_mute(): #Defines a function to toggle the mute state.
            st.session_state.mute = not st.session_state.mute #Toggles the mute state by setting it to the opposite of its current value. If it was False, it becomes True and vice versa.

        # Function to handle user question input
        def handle_user_question(user_question): #Defines a function to handle user input questions.
            if user_question: #Checks if the user_question is not empty. If it's not empty, it proceeds to process the question.
                with st.spinner("Fetching answer..."):
                    try: #Attempts to process the user's question and retrieve an answer.
                        response = user_input(user_question) #Uses st.markdown to display the response within a styled div container. The response is shown under a "Reply:" heading.
                        st.markdown(f"""
                        <div class="response">
                            <h3>Reply:</h3>
                            <p>{response}</p>
                        </div>
                        """, unsafe_allow_html=True)
                        text_to_speech(response, mute=st.session_state.mute)  # Read out the response if not muted #If mute is False, the response will be read out loud. If mute is True, the speech will be suppressed.
                    except Exception as e: #Catches any exceptions that occur during processing and displays an error message with the exception details.
                        st.error(f"An error occurred: {e}")

        # Input for user question (text)
        user_question = st.text_input("Enter your question:", key="question") #Creates a text input field for the user to enter their question.
                                                             #parameter ensures that the state of this input field is managed across reruns of the app.
        # Toggle button to mute/unmute
        if st.button("Mute" if st.session_state.mute else "Unmute", key="mute-btn", on_click=toggle_mute): # toggles between "Mute" and "Unmute"
            st.session_state.mute = not st.session_state.mute

        if st.button("Ask"):#Creates a button labeled "Ask" for submitting the text input question.
            handle_user_question(user_question)#Calls the handle_user_question function with the user's text input question when the "Ask" button is clicked.

        # Input for user question (voice)
        if st.button("Ask using Voice", key="voice-btn"): #Creates a button labeled "Ask using Voice" for submitting a voice input question.
            voice_command = get_voice_command() #Calls the get_voice_command function to capture and transcribe the user's voice command.
            if voice_command: #Checks if a valid voice command was captured.
                handle_user_question(voice_command) #Calls the handle_user_question function with the transcribed voice command when the "Ask using Voice" button is clicked.

if __name__ == "__main__": #Ensures that the main function is called when the script is executed, initializing the Streamlit application.
    main()

In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
import docx
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import pyttsx3
import speech_recognition as sr
import openpyxl

load_dotenv()

# Function to extract text from PDF files
@st.cache_data(show_spinner=False)
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# Function to extract text from Word files
@st.cache_data(show_spinner=False)
def get_docx_text(docx_files):
    text = ""
    for doc in docx_files:
        doc_reader = docx.Document(doc)
        for para in doc_reader.paragraphs:
            text += para.text + "\n"
    return text

# Function to extract text from Excel files
@st.cache_data(show_spinner=False)
def get_excel_text(excel_files):
    text = ""
    for excel in excel_files:
        df = pd.read_excel(excel)
        text += df.to_string(index=False)
    return text

# Function to split text into chunks
@st.cache_data(show_spinner=False)
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

# Function to create vector store using FAISS
@st.cache_data(show_spinner=False)
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

# Function to load conversational chain for question answering
def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context. If the answer is not in
    the provided context, just say, "Answer is not available in the context." Do not provide a wrong answer.\n\n
    Context:\n {context}\n
    Question: \n{question}\n
    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

# Function to process user input question and retrieve answer
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)
    chain = get_conversational_chain()
    response = chain.invoke({"input_documents": docs, "question": user_question}, return_only_outputs=True)
    return response["output_text"]

# Function to convert text to speech using pyttsx3
def text_to_speech(text, mute=False):
    if not mute:
        engine = pyttsx3.init()
        engine.setProperty('rate', 150)  # Speed percent (can go over 100)
        engine.setProperty('volume', 1.0)  # Volume 0-1
        engine.say(text)
        engine.runAndWait()

# Function to handle voice commands
def get_voice_command():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    with microphone as source:
        st.info("Listening for your question...")
        audio = recognizer.listen(source)
    try:
        command = recognizer.recognize_google(audio)
        st.success(f"You said: {command}")
        return command
    except sr.UnknownValueError:
        st.error("Sorry, I did not understand that.")
        return ""
    except sr.RequestError:
        st.error("Could not request results; check your network connection.")
        return ""

# Main Streamlit application
def main():
    st.set_page_config(page_title="Document Chatbot", layout="wide")
    st.title("Document Chatbot")

    st.markdown(
        """
        <style>
        .main {
            background-color: #000000;
            padding: 2rem;
            font-family: Arial, sans-serif;
            color: #333;
        }
        .sidebar .sidebar-content {
            background-image: linear-gradient(#2E7D32,#1B5E20);
            color: white;
        }
        .stButton>button {
            color: white;
            background-color: #2E7D32;
            border: None;
            padding: 0.5rem 1rem;
            font-size: 1rem;
            border-radius: 8px;
        }
        .stButton>button:hover {
            background-color: #1B5E56;
            border: None;
        }
        .header {
            text-align: center;
            margin-bottom: 2rem;
        }
        .header h1 {
            font-size: 3rem;
            color: #2E7D32;
        }
        .header p {
            font-size: 1.2rem;
            color: #666;
        }
        .container {
            display: flex;
            flex-direction: column;
            align-items: center;
        }
        .question-input {
            width: 80%;
            margin-bottom: 1rem;
        }
        .response {
            width: 80%;
            padding: 1rem;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            margin-top: 1rem;
        }
        .response h3 {
            color: #2E7D32;
        }
        .speak-btn {
            background-color: #2E7D32; /* Green color */
            color: white;
            border: none;
            padding: 0.5rem 1rem;
            font-size: 1rem;
            border-radius: 8px;
            margin-top: 1rem;
        }
        .speak-btn:hover {
            background-color: #1B5E56; /* Darker green on hover */
        }
        .mute-btn {
            background-color: #FF5722; /* Orange color */
            color: white;
            border: none;
            padding: 0.5rem 1rem;
            font-size: 1rem;
            border-radius: 8px;
            margin-top: 1rem;
        }
        .mute-btn:hover {
            background-color: #E64A19; /* Darker orange on hover */
        }
        </style>
        """,
        unsafe_allow_html=True
    )

    st.markdown("""
    <div class="header">
        <h1> Chat with Dexter 🤖</h1>
        <p>Upload your documents, process them, and ask questions directly from the content.</p>
    </div>
    """, unsafe_allow_html=True)

    # File uploader for various document types
    doc_files = st.file_uploader("Upload your Documents", accept_multiple_files=True, type=["pdf", "docx", "xlsx"])

    if doc_files:
        # Process documents and store vector store
        if st.button("Submit & Process"):
            with st.spinner("Processing..."):
                try:
                    raw_text = ""
                    pdf_docs = [doc for doc in doc_files if doc.type == "application/pdf"]
                    docx_files = [doc for doc in doc_files if doc.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document"]
                    excel_files = [doc for doc in doc_files if doc.type == "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"]

                    if pdf_docs:
                        raw_text += get_pdf_text(pdf_docs)
                    if docx_files:
                        raw_text += get_docx_text(docx_files)
                    if excel_files:
                        raw_text += get_excel_text(excel_files)

                    text_chunks = get_text_chunks(raw_text)
                    get_vector_store(text_chunks)
                    st.success("Processing complete. You can now ask questions!")
                except Exception as e:
                    st.error(f"An error occurred: {e}")

        # State management for mute functionality
        if "mute" not in st.session_state:
            st.session_state.mute = False

        def toggle_mute():
            st.session_state.mute = not st.session_state.mute

        # Function to handle user question input
        def handle_user_question(user_question):
            if user_question:
                with st.spinner("Fetching answer..."):
                    try:
                        response = user_input(user_question)
                        st.markdown(f"""
                        <div class="response">
                            <h3>Reply:</h3>
                            <p>{response}</p>
                        </div>
                        """, unsafe_allow_html=True)
                        text_to_speech(response, mute=st.session_state.mute)  # Read out the response if not muted
                    except Exception as e:
                        st.error(f"An error occurred: {e}")

        # Input for user question (text)
        user_question = st.text_input("Enter your question:", key="question")

        # Toggle button to mute/unmute
        if st.button("Mute" if st.session_state.mute else "Unmute", key="mute-btn", on_click=toggle_mute):
            st.session_state.mute = not st.session_state.mute

        if st.button("Ask"):
            handle_user_question(user_question)

        # Input for user question (voice)
        if st.button("Ask using Voice", key="voice-btn"):
            voice_command = get_voice_command()
            if voice_command:
                handle_user_question(voice_command)

if __name__ == "__main__":
    main()